In [1]:
from bs4 import BeautifulSoup as bs
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
from selenium import webdriver
from urllib.parse import urlencode
import json
import os
import pandas as pd
import re
import requests
import time

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [3]:
region_map = {
    '台北市':'1',
    '基隆市':'2',
    '新北市':'3',
    '新竹市':'4',
    '新竹縣':'5',
    '桃園市':'6',
    '苗栗縣':'7',
    '台中市':'8',
    '彰化縣':'10',
    '南投縣':'11',
    '嘉義市':'12',
    '嘉義縣':'13',
    '雲林縣':'14',
    '台南市':'15',
    '高雄市':'17',
    '屏東縣':'19',
    '宜蘭縣':'21',
    '台東縣':'22',
    '花蓮縣':'23',
    '澎湖縣':'24',
    '金門縣':'25',
    '連江縣':'26',
}

In [4]:
client = requests.session()

In [42]:
def get_var(client,region,headers):
    index_url='https://rent.591.com.tw/?kind=0&regionid='+region_map[region]
    res = client.get(index_url,headers=headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp']= region_map[region]
#     print(res.cookies.get_dict())
    soup = bs(res.text, "lxml")
#     print(soup.select('head > meta'))

    totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']
    CSRF_TOKEN = [i['content'] for i in soup.select('head > meta[name]') if i['name']=="csrf-token"][0]
    return (totalRows ,CSRF_TOKEN)

In [45]:
def get_searchUrl(i,region,headers):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region_map[region],
        'firstRow=':30*(i+1),
        'totalRows':get_var(client,region,headers)[0]
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
    url = base_url + urlencode(params)
    return url

In [46]:
get_searchUrl(0,'新北市',headers)

'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow%3D=30&totalRows=9316'

In [55]:
from pprint import pprint

In [ ]:
'https://rent.591.com.tw/'

In [72]:
def get_searchUrlist(client,region,headers):
    vars = get_var(client,region,headers)
    headers['X-CSRF-Token'] = vars[1]
    searchUrlist=[]
    for i in range(round(60/30)):
        searchUrl = get_searchUrl(i,region,headers)
        print(i)
        res_search = client.get(searchUrl,headers=headers)
        res_search.encoding = 'utf-8'
        data = json.loads(res_search.text)
        pprint(data['data']['topData'][0])
#         return data['data']['topData'][0]
        rent_list = [(i['post_id'],'https://rent.591.com.tw/'+i['detail_url'] for i in data['data']['topData']]
        print(len(rent_list))              
        searchUrlist.extend(rent_list)
    return searchUrlist
        

SyntaxError: invalid syntax (<ipython-input-72-db29fba31740>, line 13)

In [73]:
%time searchUrlist = get_searchUrlist(client,'新北市',headers)

0
{'address': '大忠街郵局對面巷內鬧中取靜的優質套房',
 'address_2': '大忠街郵局對面巷內鬧中取靜...',
 'address_3': '大忠街郵局對面巷..',
 'alt': '新北租屋,淡水租屋,獨立套房出租,大忠街郵局對面巷內鬧中取靜的優質套房',
 'area': '5坪',
 'classLast': 'class = "last"',
 'detail_url': 'rent-detail-8891616.html',
 'ico': 'rent-ico',
 'img_src': 'https://hp2.591.com.tw/house/active/2013/12/25/138797133718597206_210x158.crop.jpg',
 'isAd': 0,
 'is_new_index': 0,
 'is_new_list': 1,
 'kind_str': '獨立套房',
 'onepxImg': '',
 'photoNum': 10,
 'post_id': 8891616,
 'price': '5,500',
 'price_str': '5,500元',
 'price_unit': '元/月',
 'recom_num': 0,
 'region': 3,
 'regionid': 3,
 'section_str': '淡水區',
 'type': 1}
1
{'address': '昌平國小旁車位優惠出租中！',
 'address_2': '昌平國小旁車位優惠出租中！...',
 'address_3': '昌平國小旁車位優..',
 'alt': '新北租屋,新莊租屋,車位出租,昌平國小旁車位優惠出租中！',
 'area': '8坪',
 'classLast': 'class = "last"',
 'detail_url': 'rent-detail-8804680.html',
 'ico': 'rent-ico',
 'img_src': 'https://hp2.591.com.tw/house/active/2018/03/07/152041236215158600_210x158.crop.jpg',
 'isAd': 0,
 'is_new_index': 0,
